In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test_hw05_question4') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/23 08:30:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/23 08:30:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql import types

In [3]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True),
    types.StructField('PUlocationID', types.IntegerType(), True),
    types.StructField('DOlocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [4]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv')

In [5]:
from pyspark.sql import functions as F

In [15]:
df = df.withColumn('hours_diff', (F.unix_timestamp(df.dropOff_datetime) - F.unix_timestamp(df.pickup_datetime)) / 3600)

In [16]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|          hours_diff|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|                 0.2|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013| 0.03138888888888889|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014| 0.42694444444444446|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|0.021666666666666667|
|            

In [22]:
# df.groupby().max('hours_diff').first().asDict()
longest_trip = df.groupby().max('hours_diff').first().asDict()['max(hours_diff)']

{'max(hours_diff)': 631152.5}

In [21]:
longest_trip

631152.5

In [17]:
df.createOrReplaceTempView("fhv_trips")

In [18]:
longest_trip = spark.sql("SELECT MAX(hours_diff) AS longest_trip FROM fhv_trips").first().asDict()['longest_trip']


In [19]:
longest_trip

631152.5